In [1]:
import numpy as np
import pandas as pd
import math

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.max_rows = None  # 모든 row 내용 print
pd.options.display.max_rows = 20

### Load the dataset with pandas
The car data set we will be using in this lab is provided as a comma separated file without a header row.  In order for each column to have a meaningful header name we must provide it.  We get the information about the columns from the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile).

We will use the features of the car, to try to predict its price.

In [3]:
# Provide the names for the columns since the CSV file with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

In [4]:
# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                     sep=',', names=feature_names, header=None, encoding='latin-1')

 We'll then randomize the data, just to be sure not to get any pathological ordering effects that might harm the performance of Stochastic Gradient Descent.

In [5]:
car_data = car_data.reindex(np.random.permutation(car_data.index))
# or equivalently,
# car_data = car_data.sample(frac=1)

In [6]:
print("Data set loaded. Num examples: ", len(car_data))

Data set loaded. Num examples:  205


In [7]:
car_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
178,3,197,toyota,gas,std,two,hatchback,rwd,front,102.90,...,171,mpfi,3.27,3.35,9.30,161,5200,20,24,16558
44,1,?,isuzu,gas,std,two,sedan,fwd,front,94.50,...,90,2bbl,3.03,3.11,9.60,70,5400,38,43,?
128,3,?,porsche,gas,std,two,convertible,rwd,rear,89.50,...,194,mpfi,3.74,2.90,9.50,207,5900,17,25,37028
158,0,91,toyota,diesel,std,four,sedan,fwd,front,95.70,...,110,idi,3.27,3.35,22.50,56,4500,34,36,7898
190,3,256,volkswagen,gas,std,two,hatchback,fwd,front,94.50,...,109,mpfi,3.19,3.40,8.50,90,5500,24,29,9980


This is a really small dataset! Only 205 examples.

For simplicity in this codelab, we do not split the data further into training and validation. But you MUST do this on real datasets, or else you will overfit to your single dataset.

## Task 0: Use pandas to explore and prepare the data

- Use Pandas to inspect the data and manually curate a list of numeric_feature_names and categorical_feature_names.


Useful functions:
- `type()` called on any Python object describes the type of the object
- `dataframe[4:7]` pulls out rows 4, 5, 6 in a Pandas dataframe
- `dataframe[['mycol1', 'mycol2']]` pulls out the two requested columns into a new Pandas dataframe
- `dataframe['mycol1']` returns a Pandas series -- not a dataframe!
- `dataframe.describe()` prints out statistics for each dataframe column

In [8]:
car_data[4:7]

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
190,3,256,volkswagen,gas,std,two,hatchback,fwd,front,94.50,...,109,mpfi,3.19,3.40,8.50,90,5500,24,29,9980
112,0,161,peugot,diesel,turbo,four,sedan,rwd,front,107.90,...,152,idi,3.70,3.52,21.00,95,4150,28,33,16900
166,1,168,toyota,gas,std,two,hatchback,rwd,front,94.50,...,98,mpfi,3.24,3.08,9.40,112,6600,26,29,9538


In [9]:
car_data.describe()

,symboling,wheel-base,length,width,height,weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,10.14,25.22,30.75
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,3.97,6.54,6.89
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,7.00,13.00,16.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,8.60,19.00,25.00
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,9.00,24.00,30.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,9.40,30.00,34.00
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,23.00,49.00,54.00


In [10]:
LABEL = 'price'

numeric_feature_names = ['symboling', 'normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio']

categorical_feature_names = list(set(feature_names) - set(numeric_feature_names) - set([LABEL]))

categorical_feature_names

assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [11]:
# The correct solution will pass these assert statements.
assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [12]:
# Run to inspect numeric features.
car_data[numeric_feature_names]

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio
178,3,197,102.90,183.50,67.70,52.00,2976,171,161,5200,20,24,3.27,3.35,9.30
44,1,?,94.50,155.90,63.60,52.00,1874,90,70,5400,38,43,3.03,3.11,9.60
128,3,?,89.50,168.90,65.00,51.60,2800,194,207,5900,17,25,3.74,2.90,9.50
158,0,91,95.70,166.30,64.40,53.00,2275,110,56,4500,34,36,3.27,3.35,22.50
190,3,256,94.50,165.70,64.00,51.40,2221,109,90,5500,24,29,3.19,3.40,8.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,3,194,91.30,170.70,67.90,49.70,3071,181,160,5200,19,25,3.43,3.27,9.00
15,0,?,103.50,189.00,66.90,55.70,3230,209,182,5400,16,22,3.62,3.39,8.00
131,2,?,96.10,176.80,66.60,50.50,2460,132,?,?,23,31,3.46,3.90,8.70
3,2,164,99.80,176.60,66.20,54.30,2337,109,102,5500,24,30,3.19,3.40,10.00


In [13]:
car_data[numeric_feature_names].describe()

,symboling,wheel-base,length,width,height,weight,engine-size,city-mpg,highway-mpg,compression-ratio
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,25.22,30.75,10.14
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,6.54,6.89,3.97
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,13.00,16.00,7.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,19.00,25.00,8.60
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,24.00,30.00,9.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,30.00,34.00,9.40
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,49.00,54.00,23.00


In [14]:
# Coerce the numeric features to numbers. This is necessary because the model crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
    car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors ='coerce')

"""
errors: error는 총 3개의 옵션이 존재합니다.
- errors = 'ignore' -> 만약 숫자로 변경할 수 없는 데이터라면 숫자로 변경하지 않고 원본 데이터를 그대로 반환합니다.
- errors = 'coerce' -> 만약 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 설정하여 반환합니다.
- errors = 'raise' -> 만약 숫자로 변경할 수 없는 데이터라면 에러를 일으키며 코드를 중단합니다."""
    
# Fill missing values with 0.
# Is this an OK thing to do? You may want to come back and revisit this decision later.
car_data.fillna(0, inplace=True)

## Task 1: Make your best model with numeric features. No normalization allowed.

Modify the model provided below to achieve the lowest eval loss. You may want to change various hyperparameters:
- learning rate
- choice of optimizer
- hidden layer dimensions -- make sure your choice here makes sense given the number of training examples
- batch size
- num training steps
- (anything else you can think of changing)

Do not use the `normalizer_fn` arg on `numeric_column`.

In [15]:
car_data.shape

(205, 26)

In [16]:
X = car_data[numeric_feature_names].copy()
y = car_data['price'].copy()

In [17]:
X.columns

Index(['symboling', 'normalized-losses', 'wheel-base', 'length', 'width',
       'height', 'weight', 'engine-size', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'bore', 'stroke', 'compression-ratio'],
      dtype='object')

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=1, input_shape=(15,), activation='relu'))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [20]:
batch_size=16
epochs=20
model.fit(X_train, y_train, epochs=epochs, batch_size= batch_size)

Epoch 1/20
11/11 [==============================] - 0s 563us/step - loss: 246205216.0000 - mean_squared_error: 246205216.0000
Epoch 2/20
11/11 [==============================] - 0s 542us/step - loss: 246189712.0000 - mean_squared_error: 246189712.0000
Epoch 3/20
11/11 [==============================] - 0s 515us/step - loss: 246181808.0000 - mean_squared_error: 246181808.0000
Epoch 4/20
11/11 [==============================] - 0s 566us/step - loss: 246172000.0000 - mean_squared_error: 246172000.0000
Epoch 5/20
11/11 [==============================] - 0s 496us/step - loss: 246159440.0000 - mean_squared_error: 246159440.0000
Epoch 6/20
11/11 [==============================] - 0s 499us/step - loss: 246149888.0000 - mean_squared_error: 246149888.0000
Epoch 7/20
11/11 [==============================] - 0s 399us/step - loss: 244239856.0000 - mean_squared_error: 244239856.0000
Epoch 8/20
11/11 [==============================] - 0s 496us/step - loss: 224455712.0000 - mean_squared_error: 2244557

In [21]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 29008328.0000 - mean_squared_error: 29008328.0000


[29008328.0, 29008328.0]